In [1]:
import pandas as pd
import numpy as np
import pdb
import time

import sys
sys.path.append("../utils/")
import token_util
pro = token_util.set_token()

*获取股票列表*

In [2]:
stock_list = pd.read_csv("../data_pulled/stock_date_delta_priceNone.csv")
print("stock_list.shape: ", stock_list.shape)
print("")
stock_list.head()

stock_list.shape:  (770, 4)



,Unnamed: 0,ts_code,name,total_mv
0,0,000002.SZ,万科A,3.009761e+07
1,1,000012.SZ,南玻A,1.370715e+06
2,2,000016.SZ,深康佳A,2.090097e+06
3,3,000021.SZ,深科技,2.551164e+06
4,4,000027.SZ,深圳能源,2.128932e+06


In [2]:
# 获取市值大于100亿的股票

# for i in range(len(stock_list)):
    
#     try:
        
#         df = pro.daily_basic(ts_code=stock_list["ts_code"][i], trade_date='20200401', fields='ts_code,total_mv')
#         stock_list.loc[i, "total_mv"] = df.loc[0, "total_mv"] if df.empty is False else 0
        
#     except:
#         time.sleep(3)

# stock_list = stock_list[stock_list["total_mv"] > 1000000].reset_index(drop=True)
# stock_list.to_csv("growth_stock_list_100.csv", index=False)

### 成长股
某季度净利润增长超过150%<br>
市值大于100亿<br>
某季度和对应的上季度未出现亏损

In [3]:
# 判断某股票某段时间内的净利润增长率是否符合要求
def judge_profit_yoy(ts_code, start_fina_indicator, start_income, end, profit_threshold):
    
    try:
        profit_ok = False

        ret = pro.fina_indicator(ts_code=ts_code, start_date=start_fina_indicator, \
                                 end_date=end, fields='ts_code,end_date,q_profit_yoy')
        for q_profit in ret["q_profit_yoy"]:
            if q_profit > profit_threshold:
                ret = pro.income(ts_code=ts_code, start_date=start_income, end_date=end, \
                         fields='ts_code,end_date,n_income')
                ret = ret.drop_duplicates().reset_index(drop=True)
                for n_income in ret["n_income"]:
                    if n_income > 0:
                        continue
                    else:
                        break
                        
                profit_ok = True
                
            else:
                continue

    except:
        time.sleep(3)
            
    return profit_ok

*进行筛选*

In [4]:
candidate_list = []

for stock_code in stock_list["ts_code"]:
    
    profit_ok = judge_profit_yoy(stock_code, '20190101', '20180101', '20191230', 150)
    if profit_ok:
        candidate_list.append(stock_code)

In [5]:
len(candidate_list)

134

In [6]:
candidate_list

['000027.SZ',
 '000061.SZ',
 '000301.SZ',
 '000400.SZ',
 '000403.SZ',
 '000415.SZ',
 '000539.SZ',
 '000656.SZ',
 '000690.SZ',
 '000710.SZ',
 '000712.SZ',
 '000732.SZ',
 '000785.SZ',
 '000807.SZ',
 '000810.SZ',
 '000877.SZ',
 '000958.SZ',
 '000987.SZ',
 '000999.SZ',
 '001872.SZ',
 '002024.SZ',
 '002028.SZ',
 '002124.SZ',
 '002151.SZ',
 '002157.SZ',
 '002180.SZ',
 '002194.SZ',
 '002216.SZ',
 '002230.SZ',
 '002250.SZ',
 '002252.SZ',
 '002291.SZ',
 '002299.SZ',
 '002324.SZ',
 '002353.SZ',
 '002399.SZ',
 '002409.SZ',
 '002414.SZ',
 '002429.SZ',
 '002439.SZ',
 '002458.SZ',
 '002463.SZ',
 '002481.SZ',
 '002498.SZ',
 '002594.SZ',
 '002600.SZ',
 '002605.SZ',
 '002625.SZ',
 '002647.SZ',
 '002683.SZ',
 '002714.SZ',
 '002797.SZ',
 '300001.SZ',
 '300012.SZ',
 '300014.SZ',
 '300017.SZ',
 '300115.SZ',
 '300118.SZ',
 '300168.SZ',
 '300212.SZ',
 '300223.SZ',
 '300236.SZ',
 '300251.SZ',
 '300352.SZ',
 '300363.SZ',
 '300379.SZ',
 '300498.SZ',
 '300502.SZ',
 '300602.SZ',
 '300682.SZ',
 '300702.SZ',
 '3007

In [ ]:
stock_list[(stock_list["ts_code"] == stock_code)]